# Reactions
This notebook contains all the steps followed in order to reconcile the existing metabolic reconstruction in CHO cells. It si divided into two parts: **1. Network Reconstruction** and **2. Identification of duplicated reactions**

[1. Network Reconstruction](#reconstruction) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Datasets generation and merge**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Normalization of the data** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Group all the data into a unified dataset** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets** <br>

[2. Identification of Duplicated Reactions](#duplicated) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

[3. Add BiGG database links to every reaction](#bigg) <br>

[4. Divide into compartments](#compartments) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

## 1. Network Reconstruction <a id='reconstruction'></a>
These include the generation of datasets (1), normalization of the data (2), merging all four reconstructions into a unified dataset (3), compiling the generated reconstruction into a cobra model in order to identify duplicated reactions (4), fixing duplicated reactions using BiGG ID annotation (5), adding Recon3D GPR information (6), and finally dividing the dataset into two different datasets that will be further curated in Google Sheets (7).

In [ ]:
# Libraries import
import pandas as pd
import numpy as np

### 1.1 Datasets generation and merge
Dataset generation from previous reconstructions (CHO_DG44, iCHO1766, iCHO2101, iCHO2291)

In [ ]:
#Read excel files and create the dfs

#Camels CHO_DG44 metabolic reconstruction
camel_df = pd.read_excel('../Data/Reconciliation/datasets/CHO_DG44.xlsx', header = 1)

#Hefzi's iCHO1766 metabolic reconstruction
hefzi_df = pd.read_excel('../Data/Reconciliation/datasets/hefzi_final.xlsx')

#Foudaliha's iCHO2101 metabolic reconstruction
fouladiha_df = pd.read_excel('../Data/Reconciliation/datasets/iCHO2101.xlsx', 'Supplementary Table 10', header = 1)

#Yeo's iCHO2291 metabolic reconstruction
iCHO2291 = pd.read_excel('../Data/Reconciliation/datasets/iCHO2291_final.xlsx')

### 1.2 Normalization of the data
All dataset are normalized into the same shape and format and then combined into one big dataset

In [ ]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula', 'GPR':'GPR_fou'}, inplace = True)

# Addition of tag columns for organizational purposes
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

iCHO2291.insert(loc=0, column='cam', value=np.nan)
iCHO2291.insert(loc=1, column='hef', value=np.nan)
iCHO2291.insert(loc=2, column='fou', value=np.nan)
iCHO2291.insert(loc=3, column='yeo', value='X')

In [ ]:
# Generate cols list with the column names from all datasets
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+iCHO2291.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

In [ ]:
def add_col(df):
    '''
    This function adds the columns from the cols list 
    that are not present in the df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [ ]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
iCHO2291 = add_col(iCHO2291)
camel_df = add_col(camel_df)

In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
iCHO2291 = iCHO2291[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df['Reaction'] = camel_df['Reaction'].str.strip()

In [ ]:
# Merge all the dfs into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, iCHO2291])
all_dfs = all_dfs.reset_index(drop = True)

#Unify reaction names
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace('_cho', '')
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace(r"(e)", "_e_", regex = False)
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("[", "_")
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("]", "_")

all_dfs #20940 rows/reactions (many of them repeated)

In [ ]:
# Remove the underscore at the end of some reactions

rxns = []
idx = []

for i,row in all_dfs.iterrows():
    if str(row['Reaction']).endswith('_'):
        s = re.sub('_$', '', row['Reaction'])
        rxns.append(s)
        idx.append(i)
        
all_dfs['Reaction'].update(pd.Series(rxns,index=idx))

### 1.3 Group all the data into a unified dataset
The combined dataset generated above "all_dfs" is grouped by the Reaction BiGG ID to obtain a dataset with unique reaction identifiers

In [ ]:
# Group the data into a unified dataset

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace('[','_')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(']','')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' => | =>',' --> ')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' <-- | <--',' <=> ')
all_dfs2

In [ ]:
# Separate Demand Reaccion into a different df and remove from reconstruction. 
# Keep demand reactions from extracellular space

all_dfs2 = all_dfs2.reset_index()
demand_reactions = pd.DataFrame(columns = all_dfs2.columns)

for index, rxn in all_dfs2.iterrows():
    rxn = rxn['Reaction Formula']
    a,b = re.split('<=>|-->',rxn)
    if (b == '' or b == ' ') and not a.endswith('_e '):
        temp_df = all_dfs2[all_dfs2['Reaction Formula'] == rxn]
        demand_reactions = pd.concat([demand_reactions,temp_df])
        all_dfs2.drop(index, inplace=True)

demand_reactions = demand_reactions.reset_index(drop=True)
demand_reactions.to_excel('../Data/Reconciliation/datasets/demand_reactions.xlsx')
demand_reactions

In [ ]:
all_dfs2 = all_dfs2.reset_index(drop=True)
all_dfs2

### 1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction
The information from the dataset generated in the notebook "GPR Annotation" containing all the information from Recon3D GPRS in human and its corresponding CHO orthologs is mapped into our reconstruction

In [ ]:
# Generate a dict with recon3d reactions as keys and the CHO GPRs as values.
recon3d = pd.read_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d_dict = recon3d.set_index('m_reaction')['CHO GPR'].to_dict()
recon3d_dict

In [ ]:
# Map 'recon3d_dict' into the all_dfs3 dataframe
# the reaction IDs should be the same as those in our reconstruction

all_dfs3 = all_dfs2.reset_index()
all_dfs3['GPR_Recon3D'] = all_dfs3['Reaction'].map(recon3d_dict)

### 1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets
The dataset all_dfs3 is divided into two different datasets: all_dfs4 contains mainly all the information regarding GPRs assigned from previous reconstructions. all_dfs5 contains the rest of the attributes in the reconstruction such as EC number, bounds, etc.

In [ ]:
# all_dfs4 contains mainly information of the reactions GPRs
all_dfs4 = all_dfs3[['Reaction', 'Reaction Name', 'Reaction Formula', 'Subsystem', 'GPR_hef', 'GPR_fou', 'GPR_yeo', 'GPR_Recon3D', 'Curation Notes', 'References']]
all_dfs4.insert(8,'GPR_final', '')
all_dfs4.to_excel('../Data/Reconciliation/datasets/all_dfs4.xlsx')
all_dfs4

In [ ]:
# all_dfs5 contains information of the rest of the attributes in our reconstruction
all_dfs5 = all_dfs3[['Reaction', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible', 'Lower bound', 'Upper bound', 'Objective']]
all_dfs5.to_excel('../Data/Reconciliation/datasets/all_dfs5.xlsx')
all_dfs5

## 2. Identifications of Duplicated Reactions <a id='duplicated'></a>
In this part of the notebook we use the **duplicated_reactions** fucntion from the **utils** module to spot duplicated reactions in our dataset. First we generate a model, using cobprapy, with our dataset, then we apply the **duplicated_reactions** function to the model, and finally we standarize the name of the duplicated reactions according to the nomenclature used in BiGG.

In [ ]:
# Libraries import
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from tqdm.notebook import tqdm
from cobra import Model, Reaction, Metabolite

from utils import duplicated_reactions
from google_sheet import GoogleSheet

### 2.1 Generate the model and identify duplicated reactions
Generation of a cobra model from the **Google Sheet dataset**. This model will be used to identify duplicated reactions from the stoichiometric matrix of the model.

In [ ]:
# Generate the necessary datasets for the identification of the duplicated reactions
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'

reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

In [ ]:
##### ----- Create a model from the reactions sheet ----- #####
model = Model("iCHO")
lr = []
for _, row in reactions.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

In [ ]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(reactions['Reaction Formula'][i])
    r.name = reactions['Reaction Name'][i]
    r.subsystem = reactions['Subsystem'][i]
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])

In [ ]:
duplicated_reactions = duplicated_reactions(model)
duplicated_reactions

### 2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above
Duplicated reactions are itereated in a for loop and mapped in the original dataset. A request is made in the 
BiGG database http://bigg.ucsd.edu with each of the duplicated reactions. If any of the duplicated reactions is in BiGG, the other reaction automatically changes its name to the one located in Bigg. This way we unifiy the names of our reactions to those in BiGG.

In [ ]:
i = 0
for rxn in tqdm(duplicated_reactions):
    
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[0]])
    if response.status_code == 200:
        if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
            print(f'1 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present in CHOv1 model')
            reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
            rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
            i += 1
    
    elif response.status_code != 200:
        response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[1]])
        if response.status_code == 200:
            if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                print(f'2 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present in CHOv1 model')
                reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                i += 1
            
        elif response.status_code != 200:
            response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[0]])
            if response.status_code == 200:
                if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                    print(f'3 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present BiGG database')
                    reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                    rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                    i += 1
                
            elif response.status_code != 200:
                response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[1]])
                if response.status_code == 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'4 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present BiGG database')
                        reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                        rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                        i += 1
                elif response.status_code != 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'5 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} not present in Bigg DB')
                        reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                        rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                        i += 1

print(len(duplicated_reactions))
print(i)

In [ ]:
# Store the original column order
column_order_rxns = reactions.columns.tolist()
column_order_att = rxns_attributes.columns.tolist()

# Group by 'BiGG ID' and keep the first non-null value in each group, then reset the index
reactions = reactions.groupby('Reaction').first().reset_index()
rxns_attributes = rxns_attributes.groupby('Reaction').first().reset_index()

# Rearrange the columns to the original order
reactions = reactions[column_order_rxns]
rxns_attributes = rxns_attributes[column_order_att]

In [ ]:
##############################################################
#### ---------------------------------------------------- ####
#### ---- Update Rxns and  Attributes Google Sheets ----- ####
#### ---------------------------------------------------- ####
##############################################################
sheet.update_google_sheet(sheet_rxns, reactions)
sheet.update_google_sheet(sheet_attributes, rxns_attributes)
print("Google Sheet updated.")

In [7]:
# Check that Reactions IDs and formulas are equal in "Rxns" and "Attributes" sheets
reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

rxnsIDseq = list(reactions['Reaction']) == list(rxns_attributes['Reaction'])
if rxnsIDseq:
    print('Reaction IDs in the Rxns and Attributes Sheets are equal\n')
else:
    rxns_sheet_ids = set(list(reactions['Reaction']))
    attr_sheet_ids = set(list(rxns_attributes['Reaction']))
    print(f'Reaction IDs that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction IDs that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

rxnsforseq = list(reactions['Reaction Formula']) == list(rxns_attributes['Reaction Formula'])
if rxnsforseq:
    print('Reaction Formulas in the Rxns and Attributes Sheets are equal')
else:
    rxns_sheet_forms = set(list(reactions['Reaction Formula']))
    attr_sheet_forms = set(list(rxns_attributes['Reaction Formula']))
    print(f'Reaction formulas that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction formulas that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

Reaction IDs in the Rxns and Attributes Sheets are equal

Reaction Formulas in the Rxns and Attributes Sheets are equal


## 3. Add BiGG database links to every reaction <a id='bigg'></a>
Here we identify which reactions from our dataset are located in BiGG and we add a direct link to that reaction in our dataset

In [1]:
# Libraries import
import time
import requests
import pandas as pd
from requests_html import HTMLSession
from bs4 import BeautifulSoup

from utils import duplicated_reactions
from google_sheet import GoogleSheet

In [2]:
def fetch_url(url, max_retries=5):
    session = HTMLSession()
    retries = 0
    while retries < max_retries:
        try:
            response = session.get(url)
            if response.status_code == 200:
                ec_numbers = extract_ec_numbers(response)
                return response,ec_numbers
            elif response.status_code != 200:
                response = None
                ec_numbers = None
                return response,ec_numbers
            else:
                retries += 1
        except requests.exceptions.RequestException as e:
            print(f"Error occurred: {e}, retrying...")
            retries += 1
            time.sleep(2)  # wait for 2 seconds before next retry
            response = None
            ec_numbers = None
    return response,ec_numbers

def extract_ec_numbers(response):
    # Initialize BeautifulSoup with the response content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements containing EC Numbers
    ec_number_elements = soup.find_all('a', href=lambda x: x and 'ec-code' in x)
    
    if len(ec_number_elements) == 0:
        print('No EC Number asociated with this rxns in BiGG')
        return None
    
    elif len(ec_number_elements) > 0:
        # Create a list to hold the EC Numbers and their links
        ec_numbers = []
        ec_links = []

        for element in ec_number_elements:
            # Extract the EC Number from the text of the element
            ec_number = element.text.strip()
            ec_numbers.append(ec_number)
            # Extract the link from the href attribute of the element
            link = element['href']
            ec_links.append(link)
        
        ec_numbers_string = ', '.join(ec_numbers)
        ec_links_string = ', '.join(ec_links)

        return ec_numbers_string, ec_links_string

In [3]:
# Generate the necessary datasets
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'

reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

In [4]:
in_chov1=0
in_bigg=0
not_bigg=0

for i,row in reactions.iterrows():
    rxn=row['Reaction']
    response_esp, ec_data_esp = fetch_url('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn)
    
    if response_esp is not None:
        print(rxn,'in CHOv1')
        reactions.loc[i, 'BiGG database'] = 'http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn
        if rxns_attributes.loc[i,'EC Number']=='':
            if ec_data_esp:
                rxns_attributes.loc[i,'EC Number'] = ec_data_esp[0]
                rxns_attributes.loc[i,'EC Number Link'] = ec_data_esp[1]
                print(f'{rxn} EC Number found: {ec_data_esp[0]}, {ec_data_esp[1]}')
        in_chov1+=1
        
    else:
        response_gen, ec_data_gen = fetch_url('http://bigg.ucsd.edu/universal/reactions/'+rxn)
        if response_gen is not None:
            print(rxn,'in Bigg')
            reactions.loc[i, 'BiGG database'] = 'http://bigg.ucsd.edu/universal/reactions/'+rxn
            if rxns_attributes.loc[i,'EC Number']=='':
                if ec_data_gen:
                    rxns_attributes.loc[i,'EC Number'] = ec_data_gen[0]
                    rxns_attributes.loc[i,'EC Number Link'] = ec_data_gen[1]
                    print(f'{rxn} EC Number found: {ec_data_gen[0]}, {ec_data_gen[1]}')
            in_bigg+=1
        else:
            print(rxn,'not in Bigg')
            not_bigg+=1


No EC Number asociated with this rxns in BiGG
10FTHF5GLUtl in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHF5GLUtm in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHF6GLUtl in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHF6GLUtm in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHF7GLUtl in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHF7GLUtm in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHFtl in CHOv1
No EC Number asociated with this rxns in BiGG
10FTHFtm in CHOv1
No EC Number asociated with this rxns in BiGG
11DOCRTSLte in Bigg
No EC Number asociated with this rxns in BiGG
11DOCRTSLtm in CHOv1
No EC Number asociated with this rxns in BiGG
11DOCRTSLtr in CHOv1
No EC Number asociated with this rxns in BiGG
11DOCRTSTRNte in Bigg
No EC Number asociated with this rxns in BiGG
11DOCRTSTRNtm in CHOv1
No EC Number asociated with this rxns in BiGG
11DOCRTSTRNtr in CHOv1
No EC Number asociated with this rxns in BiGG
12DHCHOLabc 

3HAD81n3p not in Bigg
3HAD90m not in Bigg
3HADDMHPm not in Bigg
3HADDMNm not in Bigg
No EC Number asociated with this rxns in BiGG
3HADICOAx in Bigg
3HADMPm not in Bigg
No EC Number asociated with this rxns in BiGG
3HADPACte in Bigg
No EC Number asociated with this rxns in BiGG
3HADPACtxc in Bigg
No EC Number asociated with this rxns in BiGG
3HADPACx in Bigg
3HADPRISTp not in Bigg
3HADTMHDp not in Bigg
3HADTMTDp not in Bigg
3HAO in CHOv1
3HAPD100m not in Bigg
3HAPD120m not in Bigg
3HAPD140m not in Bigg
3HAPD60m not in Bigg
3HAPD80m not in Bigg
3HBCDm in CHOv1
3HBCDm EC Number found: 4.2.1.55, http://identifiers.org/ec-code/4.2.1.55
3HBCOAHLm in CHOv1
No EC Number asociated with this rxns in BiGG
3HBCOARc in CHOv1
3HBCOARtc not in Bigg
No EC Number asociated with this rxns in BiGG
3HCO3_NAt in CHOv1
3HDH101n6em not in Bigg
3HDH102n3m not in Bigg
3HDH102n3p not in Bigg
3HDH110m not in Bigg
3HDH123n3m not in Bigg
3HDH123n3p not in Bigg
3HDH130m not in Bigg
3HDH141n7m not in Bigg
3HDH142n6

No EC Number asociated with this rxns in BiGG
4PYRDX in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONEGLCte in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONEGLCtr in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONESULT in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONESte in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONEte in CHOv1
No EC Number asociated with this rxns in BiGG
5ADTSTSTERONEtr in CHOv1
No EC Number asociated with this rxns in BiGG
5AOPtm in CHOv1
5DGLCNR in Bigg
No EC Number asociated with this rxns in BiGG
5DHFtl in CHOv1
No EC Number asociated with this rxns in BiGG
5EIPENCm in Bigg
No EC Number asociated with this rxns in BiGG
5EIPENCte in Bigg
No EC Number asociated with this rxns in BiGG
5EIPENCtm in Bigg
No EC Number asociated with this rxns in BiGG
5FTHFt2 in CHOv1
No EC Number asociated with this rxns in BiGG
5G2OXPTtx in Bigg
5HLTDL in CHOv1
No EC Number asociated with t

No EC Number asociated with this rxns in BiGG
ACETONEt2m in CHOv1
No EC Number asociated with this rxns in BiGG
ACGAGBSIDEtg in Bigg
No EC Number asociated with this rxns in BiGG
ACGAGBSIDEtl in Bigg
No EC Number asociated with this rxns in BiGG
ACGALFUCGALACGALFUC12GAL14ACGLCGALGLUSIDEte in Bigg
No EC Number asociated with this rxns in BiGG
ACGALFUCGALACGALFUC12GAL14ACGLCGALGLUSIDEtg in Bigg
No EC Number asociated with this rxns in BiGG
ACGALFUCGALACGALFUCGALACGLCGAL14ACGLCGALGLUSIDEte in Bigg
No EC Number asociated with this rxns in BiGG
ACGALFUCGALACGALFUCGALACGLCGAL14ACGLCGALGLUSIDEtg in Bigg
ACGALK in CHOv1
No EC Number asociated with this rxns in BiGG
ACGALK2 in CHOv1
No EC Number asociated with this rxns in BiGG
ACGALtlg in Bigg
No EC Number asociated with this rxns in BiGG
ACGALtly in CHOv1
ACGAM2E in CHOv1
ACGAM6PSi in Bigg
ACGAMK in CHOv1
ACGAMPM in CHOv1
ACGAMtg not in Bigg
ACGAMtl not in Bigg
No EC Number asociated with this rxns in BiGG
ACGAMtly in CHOv1
No EC Number asoci

No EC Number asociated with this rxns in BiGG
ADNK4 in CHOv1
No EC Number asociated with this rxns in BiGG
ADNt in CHOv1
No EC Number asociated with this rxns in BiGG
ADNt4 in CHOv1
No EC Number asociated with this rxns in BiGG
ADNt5 in CHOv1
No EC Number asociated with this rxns in BiGG
ADNtl in CHOv1
No EC Number asociated with this rxns in BiGG
ADNtm in CHOv1
ADOCBLtm not in Bigg
No EC Number asociated with this rxns in BiGG
ADPACDAc in CHOv1
No EC Number asociated with this rxns in BiGG
ADPACTD in CHOv1
No EC Number asociated with this rxns in BiGG
ADPCOACROT in CHOv1
No EC Number asociated with this rxns in BiGG
ADPCOAPTE in CHOv1
ADPGLC in CHOv1
ADPGLC EC Number found: 3.6.1.13, 3.6.1.21, 3.6.1.9, http://identifiers.org/ec-code/3.6.1.13, http://identifiers.org/ec-code/3.6.1.21, http://identifiers.org/ec-code/3.6.1.9
ADPMAN in CHOv1
ADPMAN EC Number found: 3.6.1.21, http://identifiers.org/ec-code/3.6.1.21
ADPRDP in CHOv1
ADPRDP EC Number found: 3.6.1.13, http://identifiers.org/ec-

...

No EC Number asociated with this rxns in BiGG
r1855 in CHOv1
No EC Number asociated with this rxns in BiGG
r1856 in CHOv1
No EC Number asociated with this rxns in BiGG
r1857 in CHOv1
No EC Number asociated with this rxns in BiGG
r1858 in CHOv1
No EC Number asociated with this rxns in BiGG
r1859 in CHOv1
No EC Number asociated with this rxns in BiGG
r1860 in CHOv1
No EC Number asociated with this rxns in BiGG
r1861 in CHOv1
No EC Number asociated with this rxns in BiGG
r1862 in CHOv1
No EC Number asociated with this rxns in BiGG
r1863 in CHOv1
No EC Number asociated with this rxns in BiGG
r1864 in CHOv1
No EC Number asociated with this rxns in BiGG
r1865 in CHOv1
No EC Number asociated with this rxns in BiGG
r1866 in CHOv1
No EC Number asociated with this rxns in BiGG
r1867 in CHOv1
No EC Number asociated with this rxns in BiGG
r1871 in CHOv1
No EC Number asociated with this rxns in BiGG
r1872 in CHOv1
No EC Number asociated with this rxns in BiGG
r1873 in CHOv1
No EC Number asociated w

No EC Number asociated with this rxns in BiGG
r1997 in CHOv1
No EC Number asociated with this rxns in BiGG
r1998 in CHOv1
No EC Number asociated with this rxns in BiGG
r1999 in CHOv1
No EC Number asociated with this rxns in BiGG
r2000 in CHOv1
No EC Number asociated with this rxns in BiGG
r2001 in CHOv1
No EC Number asociated with this rxns in BiGG
r2002 in CHOv1
No EC Number asociated with this rxns in BiGG
r2003 in CHOv1
No EC Number asociated with this rxns in BiGG
r2004 in CHOv1
No EC Number asociated with this rxns in BiGG
r2005 in CHOv1
No EC Number asociated with this rxns in BiGG
r2006 in CHOv1
No EC Number asociated with this rxns in BiGG
r2007 in CHOv1
No EC Number asociated with this rxns in BiGG
r2008 in CHOv1
No EC Number asociated with this rxns in BiGG
r2009 in CHOv1
No EC Number asociated with this rxns in BiGG
r2010 in CHOv1
No EC Number asociated with this rxns in BiGG
r2011 in CHOv1
No EC Number asociated with this rxns in BiGG
r2012 in CHOv1
No EC Number asociated w

No EC Number asociated with this rxns in BiGG
r2144 in CHOv1
No EC Number asociated with this rxns in BiGG
r2145 in CHOv1
No EC Number asociated with this rxns in BiGG
r2146 in CHOv1
No EC Number asociated with this rxns in BiGG
r2147 in CHOv1
No EC Number asociated with this rxns in BiGG
r2148 in CHOv1
No EC Number asociated with this rxns in BiGG
r2149 in CHOv1
No EC Number asociated with this rxns in BiGG
r2150 in CHOv1
No EC Number asociated with this rxns in BiGG
r2151 in CHOv1
No EC Number asociated with this rxns in BiGG
r2152 in CHOv1
No EC Number asociated with this rxns in BiGG
r2153 in CHOv1
No EC Number asociated with this rxns in BiGG
r2154 in CHOv1
No EC Number asociated with this rxns in BiGG
r2155 in CHOv1
No EC Number asociated with this rxns in BiGG
r2156 in CHOv1
No EC Number asociated with this rxns in BiGG
r2157 in CHOv1
No EC Number asociated with this rxns in BiGG
r2158 in CHOv1
No EC Number asociated with this rxns in BiGG
r2159 in CHOv1
No EC Number asociated w

No EC Number asociated with this rxns in BiGG
r2280 in CHOv1
No EC Number asociated with this rxns in BiGG
r2281 in CHOv1
No EC Number asociated with this rxns in BiGG
r2282 in CHOv1
No EC Number asociated with this rxns in BiGG
r2283 in CHOv1
No EC Number asociated with this rxns in BiGG
r2284 in CHOv1
No EC Number asociated with this rxns in BiGG
r2285 in CHOv1
No EC Number asociated with this rxns in BiGG
r2286 in CHOv1
No EC Number asociated with this rxns in BiGG
r2287 in CHOv1
No EC Number asociated with this rxns in BiGG
r2288 in CHOv1
No EC Number asociated with this rxns in BiGG
r2289 in CHOv1
No EC Number asociated with this rxns in BiGG
r2290 in CHOv1
No EC Number asociated with this rxns in BiGG
r2291 in CHOv1
No EC Number asociated with this rxns in BiGG
r2292 in CHOv1
No EC Number asociated with this rxns in BiGG
r2293 in CHOv1
No EC Number asociated with this rxns in BiGG
r2294 in CHOv1
No EC Number asociated with this rxns in BiGG
r2295 in CHOv1
No EC Number asociated w

No EC Number asociated with this rxns in BiGG
r2449 in CHOv1
No EC Number asociated with this rxns in BiGG
r2465 in CHOv1
No EC Number asociated with this rxns in BiGG
r2471 in Bigg
No EC Number asociated with this rxns in BiGG
r2472 in CHOv1
No EC Number asociated with this rxns in BiGG
r2482 in CHOv1
No EC Number asociated with this rxns in BiGG
r2483 in CHOv1
No EC Number asociated with this rxns in BiGG
r2484 in CHOv1
No EC Number asociated with this rxns in BiGG
r2485 in CHOv1
No EC Number asociated with this rxns in BiGG
r2486 in CHOv1
No EC Number asociated with this rxns in BiGG
r2487 in CHOv1
No EC Number asociated with this rxns in BiGG
r2488 in CHOv1
No EC Number asociated with this rxns in BiGG
r2489 in CHOv1
No EC Number asociated with this rxns in BiGG
r2490 in CHOv1
No EC Number asociated with this rxns in BiGG
r2491 in CHOv1
No EC Number asociated with this rxns in BiGG
r2492 in CHOv1
No EC Number asociated with this rxns in BiGG
r2493 in CHOv1
No EC Number asociated wi

In [5]:
print(f'Reactions from CHOv1 reconstruction {in_chov1}')
print(f'Reactions in BiGG database {in_bigg} [not including those in CHOv1)')
print(f'Reactions NOT in BiGG {not_bigg}')

Reactions from CHOv1 reconstruction 5330
Reactions in BiGG database 3187 [not including those in CHOv1)
Reactions NOT in BiGG 1989


In [6]:
############################################################
#### -------------------------------------------------- ####
#### ---- Update Rxns and Attributes Google Sheet ----- ####
#### -------------------------------------------------- ####
############################################################
sheet.update_google_sheet(sheet_rxns, reactions)
sheet.update_google_sheet(sheet_attributes, rxns_attributes)
print("Google Sheet updated.")

Google Sheet updated.


## 4. Divide into compartments <a id='compartments'></a>

In [ ]:
from cobra.io import load_json_model
import pandas as pd

# model = read_sbml_model('iCHOv3_CHO_05072023.xml')
model = load_json_model('iCHOv3_CHO_05072023.json')

In [ ]:
subsystem_list = []
for r in model.reactions:
    subsystem_list.append(r.subsystem)
print("Number of unique subsystems:", len(set(subsystem_list)))

In [ ]:
subsystem_counts = pd.Series(subsystem_list).value_counts().reset_index()
subsystem_counts.columns = ['Subsystem', 'RXN Counts']
subsystem_counts[subsystem_counts['RXN Counts'] == 1]


In [ ]:
subsystem_counts

In [ ]:
# TODO -Remove blocked reactions from packages
#      -Define the high-importance reactions for biomass production
#      -Rename subsystems with very few reactions